# Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier,\
                             GradientBoostingClassifier, ExtraTreesClassifier,\
                             BaggingClassifier, VotingClassifier)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import KFold

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from time import time

# Set de Entrenamiento y Test Final

In [2]:
train = pd.read_csv('csv_files/modelo_final.csv')
test_final = pd.read_csv('csv_files/test_final.csv')

In [ ]:
train['rango_edad'] = train['rango_edad'].astype('category')
train['sexo'] = train['sexo'].astype('category') 
train['nivel_estudios'] = train['nivel_estudios'].astype('category')
train['esta_estudiando'] = train['esta_estudiando'].astype('category')
train['tipo_de_trabajo'] = train['tipo_de_trabajo'].astype('category')
train['nivel_laboral'] = train['nivel_laboral'].astype('category')
train['nombre_zona'] = train['nombre_zona'].astype('category')
train['nombre_area'] = train['nombre_area'].astype('category')
train['sepostulo'] = train['sepostulo'].astype('category')

In [ ]:
test_final['rango_edad'] = test_final['rango_edad'].astype('category')
test_final['sexo'] = test_final['sexo'].astype('category') 
test_final['nivel_estudios'] = test_final['nivel_estudios'].astype('category')
test_final['esta_estudiando'] = test_final['esta_estudiando'].astype('category')
test_final['tipo_de_trabajo'] = test_final['tipo_de_trabajo'].astype('category')
test_final['nivel_laboral'] = test_final['nivel_laboral'].astype('category')
test_final['nombre_zona'] = test_final['nombre_zona'].astype('category')
test_final['nombre_area'] = test_final['nombre_area'].astype('category')

In [ ]:
# Features categóricos:
f1 = 'rango_edad'
f2 = 'sexo'
f3 = 'nivel_estudios'
f4 = 'esta_estudiando'
f5 = 'tipo_de_trabajo'
f6 = 'nivel_laboral'
f7 = 'nombre_zona'
f8 = 'nombre_area'

#features = [f1,f2,f3,f4,f5,f6,f7,f8]
features = [f1,f2,f3,f4,f5,f6]

In [ ]:
x = np.array(train[features])
y = np.array(train['sepostulo'])

x_test_final = np.array(test_final[features])


# Definición de algunas variables globales

In [4]:
id_aviso_postulante = test_final['id']

ntrain = x_train.shape[0] # Cantidad de registros totales del set de entrenamiento

ntest = x_test.shape[0] # Cantidad de registros totales del set de pruebas

SEED = 0 # Seed

NFOLDS = 5 # Cantidad de bloques a particionar para KFold

kfold = KFold(ntrain, n_folds=NFOLDS, random_state=SEED) # KFold

# Creación de la clase SklearnPadre
Lo que realizamos a continuación es la creación de una clase que posee los métodos comunes de los clasificadores de Sklearn que utilizaremos en el presente trabajo. De esta forma, ahorramos líneas de código y evitamos redundancia a la hora de utilizar los métodos.

El inicializador recibe un clasificador de Sklearn, un seed y los parámetros necesarios del clasificador en cuestión.

In [5]:
class SklearnPadre(object):
    def __init__(self, clasif, seed=0, parametros=None):
        parametros['random_state'] = seed
        self.clasif = clasif(**parametros)

    def train(self, x_train, y_train):
        self.clasif.fit(x_train, y_train)

    def predict(self, x):
        return self.clasif.predict(x)
    
    def fit(self, x, y):
        return self.clasif.fit(x,y)
    
    def feature_importances(self, x, y):
        print(self.clasif.fit(x,y).feature_importances_)

# Predicciones OOF (Out Of Fold)

El ensamble utiliza predicciones de los clasificadores base como input para el entrenamiento del modelo de 2do nivel. Sin embargo, no podemos entrenar los modelos base con la totalidad del set de entrenamiento. Esto genera el riesgo de que nuestro modelo de 1er nivel conozca el test final y overfitee cuando utilice las predicciones base.

Por ello, utilizamos la siguiente función para tomar distintos bloques del set de entrenamiento.

In [6]:
def get_oof(clasif, x_train, y_train, x_test):
    oof_train = np.zeros(ntrain)
    oof_test = np.zeros(ntest)
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kfold):
        x_train_i = x_train[train_index]
        y_train_i = y_train[train_index]
        x_test_i = x_train[test_index]

        clasif.train(x_train_i, y_train_i)

        oof_train[test_index] = clasif.predict(x_test_i)
        oof_test_skf[i, :] = clasif.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1) 

# Modelos de Primer Nivel

A continuación llevaremos a cabo nuestro primer nivel de clasificación mediante el uso de algunos clasificadores de la librería Sklearn:

- 
- 
- 
- 
- 

### Definición de los hiper-parámetros necesarios para cada algoritmo

In [7]:
# RANDOM FOREST
rf_params = {'n_estimators':500, 'max_features':'sqrt', 'max_depth':6, 'min_samples_split':2,\
             'min_samples_leaf':2, 'bootstrap':True, 'oob_score':False, 'warm_start':True}

# EXTRA TREES
et_params = {'n_estimators':500, 'max_features':'sqrt', 'max_depth':8, 'min_samples_split':2,\
             'min_samples_leaf':2, 'bootstrap':True, 'oob_score':False, 'warm_start':False}

# DECISION TREE
dt_params = {'criterion':'entropy', 'splitter':'best', 'max_depth':None, 'min_samples_split':2,\
             'min_samples_leaf':2, 'max_features':'sqrt', 'presort':True}

# GRADIENT BOOSTING
gb_params = {'learning_rate':0.1, 'n_estimators':500, 'max_depth':5, 'min_samples_split':2,\
             'min_samples_leaf':2, 'subsample':1.0, 'max_features':'sqrt', 'warm_start':False}

# BAGGING CON LOGISTIC REGRESSION
bag_params = {'base_estimator':LogisticRegression(), 'n_estimators':100, 'bootstrap':True,\
              'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

### Creación de los clasificadores según los hiper-parámetros definidos

In [8]:
random_forest = SklearnPadre(clasif=RandomForestClassifier, seed=SEED, parametros=rf_params)

extra_trees = SklearnPadre(clasif=ExtraTreesClassifier, seed=SEED, parametros=et_params)

decision_tree = SklearnPadre(clasif=DecisionTreeClassifier, seed=SEED, parametros=dt_params)

grad_boost = SklearnPadre(clasif=GradientBoostingClassifier, seed=SEED, parametros=gb_params)

bag_log_reg = SklearnPadre(clasif=BaggingClassifier, seed=SEED, parametros=bag_params)

### Output de los modelos de primer nivel

A continuación realizamos las predicciones para el set de entrenamiento y el test final con OOF. Estos resultados base serán utilizados como input en el modelo de segundo nivel.

In [9]:
t0_total = time()

# Random Forest
t0_rf = time()
rf_oof_train, rf_oof_test = get_oof(random_forest, x, y, x_test_final)
tf_rf = time() - t0_rf
print ("Tiempo de entrenamiento del random forest: %0.5f seconds." % tf_rf)

# Extra Trees
t0_et = time()
et_oof_train, et_oof_test = get_oof(extra_trees, x, y, x_test_final)
tf_et = time() - t0_et
print ("Tiempo de entrenamiento del extra trees: %0.5f seconds." % tf_et)

# Decision Tree
t0_dt = time()
dt_oof_train, dt_oof_test = get_oof(decision_tree, x, y, x_test_final)
tf_dt = time() - t0_dt
print ("Tiempo de entrenamiento del decision tree: %0.5f seconds." % tf_dt)

# Gradient Boosting
t0_gb = time()
gb_oof_train, gb_oof_test = get_oof(grad_boost, x, y, x_test_final)
tf_gb = time() - t0_gb
print ("Tiempo de entrenamiento del gradient boosting: %0.5f seconds." % tf_gb)

# Bagging con Logistic Regresion
t0_bag = time()
bag_oof_train, bag_oof_test = get_oof(bag_log_reg, x, y, x_test_final)
tf_bag = time() - t0_bag
print ("Tiempo de entrenamiento del bagging con logistic regression: %0.5f seconds." % tf_bag)

# Tiempo total de ejecución
tf_total = time() - t0_total
print ("Tiempo total de entrenamiento del modelo de 1er nivel: %0.5f seconds." % tf_total)

### Importancia de los features en base a los resultados obtenidos

rf_features = random_forest.feature_importances(x,y)

et_features = extra_trees.feature_importances(x,y)

dt_features = decision_tree.feature_importances(x,y)

gb_features = grad_boost.feature_importances(x,y)

In [3]:
# ACA HAY QUE COPIAR Y PEGAR LA SALIDA DE ARRIBA PERO EN FORMATO ARRAY

### Creación de un dataframe con la importancia de los features para cada clasificador

In [ ]:
features = np.array(features)

df_features = pd.DataFrame({'Features': features,\
                            'Random Forest': rf_features,\
                            'Extra Trees': et_features,\
                            'Decision Tree': dt_features,\
                            'Gradient Boost': gb_features,\
                            })

df_features = df_features[['Features', 'Random Forest', 'Extra Trees', 'Decision Tree', 'Gradient Boost']]

df_features.head()

Agregamos una columna contenedora del promedio de la importancia de los features de cada fila

In [ ]:
df_features['promedio'] = df_features.mean(axis=1)
df_features.head()

# Modelos de Segundo Nivel

Utilizamos las predicciones obtenidas en el modelo de primer nivel como input del siguiente modelo.

Primero, analizamos las predicciones del primer nivel para cada clasificador.

In [ ]:
predicciones_primer_nivel = pd.DataFrame({'Random Forest': rf_oof_train.ravel(),\
                                          'Extra Trees': et_oof_train.ravel(),\
                                          'Decision Tree': dt_oof_train.ravel(),\
                                          'Gradient Boosting': gb_oof_train.ravel(),\
                                          'Bagging con LogReg': bag_oof_train.ravel()\
                                          })
predicciones_primer_nivel.head()

### Concatenamos las predicciones obtenidas en el primer nivel

In [14]:
x_2do_nivel = np.concatenate((et_oof_train, rf_oof_train, dt_oof_train, gb_oof_train, bag_oof_train), axis=1)

x_test_final_2do_nivel = np.concatenate((et_oof_test, rf_oof_test, dt_oof_test, gb_oof_test, bag_oof_test), axis=1)

### Aplicamos XGBoost para el modelo de segundo nivel

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el xgboost
xgboost = xgb.XGBClassifier(learning_rate = 0.02,\
                            n_estimators= 2000,\
                            max_depth= 5,\
                            min_child_weight= 2,\
                            gamma=0.9,\                        
                            subsample=0.7,\
                            colsample_bytree=0.7,\
                            objective= 'binary:logistic',\
                            nthread= -1,\
                            scale_pos_weight=1)

# Lo entrenamos con el set de entrenamiento obtenido en el modelo de 1er nivel
xgboost.fit(x_2do_nivel, y)

# Realizamos las predicciones con el set definitivo
pred_final = xgboost.predict(x_test_final_2do_nivel)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el xgboost
xgboost = xgb.XGBClassifier(learning_rate = 0.02,\
                            n_estimators= 2000,\
                            max_depth= 5,\
                            min_child_weight= 2,\
                            gamma=0.9,\                        
                            subsample=0.7,\
                            colsample_bytree=0.7,\
                            objective= 'binary:logistic',\
                            nthread= -1,\
                            scale_pos_weight=1)

# Lo entrenamos con el set de entrenamiento obtenido en el modelo de 1er nivel
xgboost.fit(x_2do_nivel, y)

# Realizamos las predicciones con el set definitivo
pred_final_proba = xgboost.predict_proba(x_test_final_2do_nivel)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

# Submit File

#### Submit con valores binarios

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_ensamble.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [ ]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_ensamble_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")